# 文本分类挖掘

机器学习中，分类问题是基础，很多应用都可以转化为分类问题，本实验重点试验文本分类。而针对中文文本数据，往往需要进行预处理和中文分词，中文分词不准确，将造成偏差过高。因此在对社交媒体文本数据训练模型之前需要数据预处理与中文分词。

本章实验分别进行文本数据预处理、中文分词和分类。

本章实验涉及KNN、SVM等分类算法，通过不同算法的效果对比来加深对算法的理解。


## 先读取相关的EXCEL文件

**pandas.read_excel(io,sheetname=0,header=0,names=None,index_col=None,usecols=None,squeeze= False,dtype=None,engine=None,converters=None,true_values=None,false_values=None,skiprows= None,nrow=None,na_values=None,keep_default_na=True,verbose=False,parse_dates=False, date_parser=None,thousands=None,comment=None,skipfooter=0,conver_float=True, mangle_dupe_cols=True)**

    io ：字符串，xls 或 xlsx 文件路径或类文件对象。
    sheet_name ：None、字符串、整数、字符串列表或整数列表，默认值为 0。字符串用于工作表名称；整数为索引，表示工作表位置，字符串列表或整数列表用于请求多个工作表，为 None 时则获取所有的工作表。参数值如表4 所示。
    header ：指定作为列名的行，默认值为 0，即取第一行的值为列名。数据为除列名以外的数据；若数据不包含列名，则设置为 header=None。
    names ：默认值为 None，要使用的列名列表。
    index_col ：指定列为索引列，默认值为 None，索引 0 是 DataFrame 对象的行标签。
    usecols ：int、list 或字符串，默认值为 None。
    如果为 None，则解析所有列。
    如果为 int，则解析最后一列。
    如果为 list 列表，则解析列号和列表的列。
    如果为字符串，则表示以逗号分隔的 Excel 列字母和列范围列表（例如，“A ：E”或    “A，C，E ：F”），范围包括双方。
    squeeze ：布尔值，默认值为 False，如果解析的数据只包含一列，则返回一个 Series。
    dtype ：列的数据类型名称或字典，默认值为 None。例如，{ 为 'a' ：np.float64，'b' ：np.int32}。
    kiprows ：省略指定行数的数据，从第一行开始。
    skipfooter ：省略指定行数的数据，从尾部数的行开始。

![](Pic/sheet.JPG)

说明：如果设置sheet_name = None, 则在生成表格的时候是返回的是dict字典类型，

因此如果想要重新处理成pandas.DataFrame类型需要修正方法，使用pandas.concat()方法对数据进行批量的连接

**pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
    keys=None, levels=None, names=None, verify_integrity=False)**

    objs: series，dataframe或者是panel构成的序列lsit
    axis： 需要合并链接的轴，0是行，1是列
    join：连接的方式 inner，或者outer
![](Pic/2t.png)

In [25]:
import pandas as pd
import collections
df_dict0 = pd.read_excel('新冠00.xlsx', sheet_name=None, index_col=None, na_values=['NA'])
df_dict0 = collections.OrderedDict(sorted(df_dict0.items()))
df = pd.concat(df_dict0.values(), ignore_index=True)


df_dict = pd.read_excel('新冠01.xlsx', sheet_name=None, index_col=None, na_values=['NA'])
df_dict = collections.OrderedDict(sorted(df_dict.items()))
dfx = pd.concat(df_dict.values(), ignore_index=True)

dfx = pd.concat([dfx, df], ignore_index=True)
dfx

,博主昵称,博主主页,微博认证,微博达人,微博内容,发布位置,发布时间,微博地址,微博来源,转发,评论,赞
0,austin_li,https://weibo.com/1850385232?refer_flag=100103...,NaN,NaN,湖人2位球员，03tohrg30，6人3位球员，掘金1位球员，凯尔特人斯玛特确诊新冠#NBA...,NaN,2020年03月30日 23:04,https://weibo.com/1850385232/IB3qrwd6t?refer_f...,0,NaN,NaN,NaN
1,魔女妈,https://weibo.com/2410092342?refer_flag=100103...,微博会员,NaN,纽约#新冠# //@战争史研究WHS:比起一下子被敲懵圈的中国，美国和欧洲至少有两个月的提前...,NaN,2020年03月30日 22:22,https://weibo.com/2410092342/IB39gA0wY?refer_f...,2,NaN,NaN,NaN
2,万宜商务咨询,https://weibo.com/5380538219?refer_flag=100103...,微博官方认证,NaN,#泰国疫情# 井喷在即，国王却失踪了？#新冠# 疫苗有重大突破！在#新加坡公众场所不保持一米...,NaN,2020年03月30日 20:49,https://weibo.com/5380538219/IB2xvDftU?refer_f...,0,NaN,NaN,NaN
3,春天文化,https://weibo.com/6083475503?refer_flag=100103...,NaN,NaN,#新冠# 下的#日本# ，东京都知事#小池百合子# 紧急记者见面会。打响面向#夜店# 等夜间...,NaN,2020年03月30日 20:40,https://weibo.com/6083475503/IB2tL7DNM?refer_f...,0,NaN,NaN,NaN
4,Fay南半球墨尔本日记,https://weibo.com/5409075755?refer_flag=100103...,NaN,NaN,3.26据统计，澳洲疫情导致两百万人失业，这是自1933年大萧条以来失业率最高的一刻，说澳洲...,NaN,2020年03月30日 20:07,https://weibo.com/5409075755/IB2gBAtCo?refer_f...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6593,windham大叔,https://weibo.com/1684306562?refer_flag=100103...,NaN,NaN,CDC的说法是COVID19疫苗预计至少要等18个月以上#新冠#,NaN,2020年03月12日 08:27,https://weibo.com/1684306562/IydZqCKCX?refer_f...,0,NaN,NaN,NaN
6594,北美青茉,https://weibo.com/2477675414?refer_flag=100103...,微博会员,NaN,#美国疫情# 美国国家过敏和传染病研究所（N.I.A.I.D.）主任Anthony Fauc...,NaN,2020年03月12日 07:57,https://weibo.com/2477675414/IydN50FMH?refer_f...,0,NaN,NaN,NaN
6595,我是你苗姐Mia,https://weibo.com/5967006660?refer_flag=100103...,微博个人认证,NaN,"#华盛顿宣布进入紧急状态#最近的一两个星期#新冠#在美国爆发,疫情也影响到了我所在首都华盛顿...",NaN,2020年03月12日 07:25,https://weibo.com/5967006660/IydAo5vK9?refer_f...,1,NaN,NaN,NaN
6596,Cheney---Chen,https://weibo.com/2629408521?refer_flag=100103...,NaN,NaN,#新冠# 现在每天看全球人数，中国快好了，欧美严重了，啥时候能好啊？希望全球早点结束，人心惶惶,NaN,2020年03月12日 06:28,https://weibo.com/2629408521/Iydd4htDg?refer_f...,0,NaN,NaN,NaN
